In [ ]:
from grid.robot.wheeled.airgen_car import AirGenCar
airgen_car_0 = AirGenCar()

In [ ]:
from grid.model.perception.vo.vo_dpvo import DPVO

import numpy as np
client = airgen_car_0.client

# Get camera parameters
camera_info = client.simGetCameraInfo("0")
print(f"camera into : {camera_info}")

# Initialize the VO model with calibration parameters
# match this with the camera settings while creating the session
calib = np.array([128, 128, 128, 128])
vo_model = DPVO(calib)

In [ ]:
import rerun as rr
import airgen
import time

client.enableApiControl(False)  # User controls the vehicle

# Get the initial pose of the vehicle
initial_pose = client.simGetVehiclePose()
initial_position = initial_pose.position
initial_orientation = initial_pose.orientation
# Initialize current global pose
current_position = initial_position
current_orientation = initial_orientation

def apply_relative_pose(current_position, current_orientation, rel_pose):
    # Extract relative translation and rotation from the pose
    rel_east = rel_pose[0] 
    rel_down = -rel_pose[1] 
    rel_north = rel_pose[2] 
    rel_qx = rel_pose[3]
    rel_qy = rel_pose[4]
    rel_qz = rel_pose[5]
    rel_qw = rel_pose[6]

    new_position = airgen.Vector3r(x_val = current_position.x_val + rel_north,
    y_val = current_position.y_val + rel_east,
    z_val = current_position.z_val + rel_down)
    # Apply relative translation
    
    # get relative orientation (assuming radians)
    relative_orientation = airgen.Quaternionr(rel_qw, rel_qx , rel_qy, rel_qz)
    
    # Apply relative rotation by quaternion multiplication
    new_orientation = current_orientation * relative_orientation
    
    return new_position, new_orientation

#airgen uses NED coordinate system
rr.log('vo', rr.ViewCoordinates.RIGHT_HAND_Z_DOWN, static=True)
try:
    t = 0
    while True:
        # Capture RGB image from Airgen
        t += 1
        rgb_image = client.getImages("front_center", [airgen.ImageType.Scene])[0][0]
        
        # Calculate pose using VO model
        relative_pose = vo_model.run(rgb_image)
        # this pose returns x,y,z,roll,pitch,yaw
        # Apply the relative pose to the current global pose
        current_position, current_orientation = apply_relative_pose(initial_position, initial_orientation, relative_pose)

        rr.log(f'vo/position/pos_{t}', rr.Points3D(positions=[current_position.x_val, current_position.y_val, -current_position.z_val]))
        
        euler_angles = current_orientation.to_euler_angles()
        pitch = euler_angles.x_val
        roll = euler_angles.y_val
        yaw = euler_angles.z_val
        time.sleep(0.1)
        

except KeyboardInterrupt:
    pass